In [1]:
import os
import regex
import time
import pandas as pd
import numpy as np
import pickle
import psycopg2
from sqlalchemy import create_engine
from sklearn.linear_model import LogisticRegressionCV

from credentials import pmc_credentials # a dict defined in a python file in the current directory

connection_string = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(**pmc_credentials)

engine = create_engine(connection_string)


def get_mean_xval_score_for_binary_classifier(clf):
    """ The highest mean xval score across the cross-validation folds """
    r = np.where(clf.Cs_ == clf.C_)[0][0]
    return np.mean([v[r] for v in clf.scores_['1']])


In [2]:
descriptor_counts = pd.read_sql("select * from pmc_descriptor_counts where tally > 2000", con=engine)
descriptor_counts

,descriptor_id,name,tally
0,D006801,Humans,869145
1,D005260,Female,416236
2,D008297,Male,377430
3,D000818,Animals,327945
4,D000328,Adult,219809
...,...,...,...
1014,D001792,Blood Platelets,2008
1015,D017667,Adipocytes,2006
1016,D020285,Cryoelectron Microscopy,2004
1017,D017024,"Chemotherapy, Adjuvant",2003


In [3]:
def get_data_for_next_10_descriptors( starting_row, bucket=0, num_examples=100):
    descriptor_list = list(descriptor_counts[starting_row:(starting_row+10)]['descriptor_id'])
    dstr = ','.join([f"'{d}'" for d in descriptor_list ])
    sql = f"select * from fn_get_dataset_for_descriptors({bucket}, {num_examples}, {dstr})"
    df = pd.read_sql(sql, con=engine)
    df['vector'] = [eval(v) for v in df['vector']]
    df.columns = [*df.columns[0:4], *descriptor_list]
    return df

In [4]:
for batch_number in range(100):
    print(f"### Batch number {batch_number} ###")
    descriptor_models = {}
    
    starting_row = batch_number * 10
    df = get_data_for_next_10_descriptors(starting_row, bucket=0, num_examples=200)
    flag_cols = df.columns[4:14]
    
    X_train = df['vector'].tolist()
    for flag_col in flag_cols:
        y_train = df[flag_col].values
        clf = LogisticRegressionCV(cv=5, scoring='roc_auc', n_jobs=-1, max_iter=10000)
        clf.fit(X_train, y_train)
        descriptor_models[flag_col] = clf
        print(flag_col, get_mean_xval_score_for_binary_classifier(clf))

    with open(f'descriptor_models_200samples_batch{batch_number:02d}.pkl', 'wb') as f:
        pickle.dump(descriptor_models, f, pickle.HIGHEST_PROTOCOL)

### Batch number 0 ###
D006801 0.9141724477152138
D005260 0.7716954514978116
D008297 0.7692757976325661
D000818 0.947664288651662
D000328 0.7773120332827308
D008875 0.8160141799066963
D000368 0.8151357349594649
D051379 0.9303825057255389
D055815 0.7344618383885549
D000293 0.7884672496578288
### Batch number 1 ###
D002648 0.8366873022624635
D012189 0.8478552221103886
D003430 0.8212772298030661
D012307 0.783522868680707
D000369 0.7437915111543818
D016896 0.8333452709116903
D015398 0.9356663922238573
D045744 0.962535368133231
D000086382 0.8915331096225989
D011795 0.8679506694810881
### Batch number 2 ###
D011379 0.8574245113052431
D011446 0.831034013020276
D011247 0.9223192303386474
D002675 0.9031578256840069
D051381 0.8847201946397993
D004195 0.8488366116391941
D049109 0.9012592464262908
D009154 0.8863041567803034
D015415 0.7589745994081205
D013997 0.7145121571112216
### Batch number 3 ###
D008810 0.922824168713493
D007223 0.890784989456567
D000465 0.9167676958920611
D000086402 0.9057306

/home/rmhorton/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


D008954 0.8388282505675775
D005838 0.9005031889970188
D014481 0.952192238838421
D011485 0.8766088762570285
D014408 0.9218859781626666
D000900 0.9366094160834916
D000970 0.8993803150901402
D002454 0.8894908254453876
D015658 0.9411343084297605
D059467 0.8678483766983037
### Batch number 7 ###
D008175 0.9244992810088851
D015994 0.9590589302648114
D003198 0.9075320157067359
D016207 0.907148867184597
D015870 0.7434271237766101
D000595 0.8791704922070771
D009474 0.933704250189271
D017422 0.8970382500906423
D014157 0.8163441620851299
D009765 0.9293091511642022
### Batch number 8 ###
D018345 0.8760528358359141
D000367 0.8937490112836597
D002465 0.8992236976880557
D002470 0.8508346114619607
D018384 0.8798976495763029
D001426 0.9382472915261127
D018570 0.9030298953442927
D018450 0.7877684145009995
D008969 0.8902329204996466
D020022 0.888474433140425
### Batch number 9 ###
D012984 0.8843557130486867
D003924 0.9300544070975864
D008962 0.9021807868002204
D001483 0.8585192554529553
D008807 0.9407519

/home/rmhorton/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


D002908 0.9059740778817902
D005947 0.8868300372297048
D007700 0.902371021567947
D007668 0.8825029573277071
D044127 0.8934062112204737
D000069550 0.9553071820447132
### Batch number 21 ###
D012424 0.9405713200725284
D034741 0.8864207645568021
D006282 0.9393885031618513
D011024 0.9357759036869358
D001315 0.8814810565850877
D020411 0.9101050612288466
D004285 0.9211043902924791
D034622 0.8964346208091356
D001499 0.9115754551685142
D019008 0.9415280318520726
### Batch number 22 ###
D002118 0.8926651626235422
D004847 0.8723752406179012
D010455 0.8670467283964213
D001007 0.9601354146643386
D005347 0.8806323530825857
D011320 0.9681857404917237
D001794 0.9522996474503753
D005434 0.8741805313542347
D018507 0.9533335301423236
D014611 0.9571472442455402
### Batch number 23 ###
D005247 0.9458457412505876
D018515 0.9549854252660619
D005075 0.9625180660344796
D002289 0.9558305270366907
D015897 0.939331632960332
D008603 0.9752149954026794
D012441 0.9636798676398508
D009361 0.9568097391504853
D016328 0

...

D001459 0.9226113915234819
D017404 0.9235804214748622
D000072138 0.9648432766773472
### Batch number 90 ###
D006224 0.8887619877853934
D013482 0.9495193804579047
D016273 0.9496374512424335
D003470 0.9439107949423334
D001290 0.9508015773680809
D003593 0.9190045673125962


/home/rmhorton/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


D064370 0.9737609725009031
D017132 0.9536897072250785
D013318 0.9887879169620337
D005239 0.939643397860365
### Batch number 91 ###
D055118 0.9517440759074999
D016366 0.9319017777036691
D001294 0.9610248635249372
D025941 0.927387419376364
D013116 0.9492811901541227
D012038 0.9217400350712112
D010349 0.9013020951559714
D015966 0.9604520629540708
D004730 0.940077796501868
D019070 0.9089295390415646
### Batch number 92 ###
D000701 0.968199590616489
D008159 0.9745512343120313
D000081246 0.8733221186190884
D000072283 0.9189116894086201
D007109 0.8461298183079375
D006526 0.9432151908956838
D050397 0.9887551713285632
D011956 0.8367425103740838
D000069600 0.9840648661214685
D011725 0.8471234317034773
### Batch number 93 ###
D004721 0.9239892508700315
D002980 0.9854146664033598
D001854 0.9206326944180017
D003299 0.9882486334116477
D015820 0.8943186435823259
D011743 0.8752343734746987
D000072669 0.9173536363423533
D018380 0.9380170841416999
D047108 0.9409607298170706
D005919 0.9785480121595317
##

/home/rmhorton/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


D051858 0.889642521626398
D001369 0.9586400274908137
D010353 0.9642621212209095


In [ ]:
# for descriptor, clf in descriptor_models.items():
#     print(descriptor, get_mean_xval_score_for_binary_classifier(clf))